In [19]:
import minsearch

In [20]:
import json

In [21]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [22]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [24]:
#Now we want to index it
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
)

In [25]:
q = 'the course has already started, can I still enroll?'

In [26]:
index.fit(documents)

In [29]:
# from openai import OpenAI
from google import genai

In [30]:
# client = OpenAI()
client = genai.Client()

In [65]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        num_results=10
    )

    return results



In [66]:
def build_prompt(query, search_results):
    #We want the LLM to put the document content in the context of the answer
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FQA database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [67]:
def llm(prompt):
    response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
    )

    return response.text

In [68]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [69]:
query = 'how do I run kafka?'

rag(query)

'To run Kafka-related applications:\n\n*   **For Java Kafka producers/consumers/kstreams:** In the project directory, run:\n    `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`\n*   **To ensure Kafka brokers are running (if you encounter `NoBrokersAvailable` error):**\n    *   Confirm your Kafka broker Docker container is not working using `docker ps`.\n    *   Then, in the docker compose yaml file folder, run `docker compose up -d` to start all the instances.'

In [70]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects."